In [ ]:
import jinja2

tpl = jinja2.Template("""{% for idx in range(N) -%}
---
apiVersion: batch/v1
kind: Job
metadata:
  name: stats-job-{{ job }}-{{ idx }}
  namespace: processing
  labels:job
    jobgroup: stats-job-{{ job }}
    app.kubernetes.io/name: stats-job-{{ job }}
spec:
  backoffLimit: 4
  ttlSecondsAfterFinish: 1
  template:
    metadata:
      labels:
        jobgroup: stats-job-{{ job }}
    spec:
      affinity:
        nodeAffinity:
          requiredDuringSchedulingIgnoredDuringExecution:
            nodeSelectorTerms:
            - matchExpressions:
              - key: nodetype
                operator: In
                values:
                - spot
      restartPolicy: Never
      containers:
      - name: worker
        image: {{ docker }}
        imagePullPolicy: IfNotPresent
        resources:
            requests:
              memory: 120Gi
              cpu: 15890m
        env:
            - name: AWS_ACCESS_KEY_ID
              valueFrom:
                   secretKeyRef:
                       name: processing-aws-creds-stats
                       key: AWS_ACCESS_KEY_ID
            - name: AWS_SECRET_ACCESS_KEY
              valueFrom:
                   secretKeyRef:
                       name: processing-aws-creds-stats
                       key: AWS_SECRET_ACCESS_KEY
            - name: AWS_DEFAULT_REGION
              valueFrom:
                   secretKeyRef:
                       name: processing-aws-creds-stats
                       key: AWS_DEFAULT_REGION
            - name: TASK_DB_S3
              value: {{ db_uri }}
            - name: TASKS
              value: {{ start + idx}}:{{ end }}:{{N}}
        command: ['/bin/bash']
        args:
            - '-c'
            - |
                odc_libs=$(pip list | awk '/^odc-/{print $1}')
                pip install -U \\
                  --no-deps \\
                  --extra-index-url="https://packages.dea.ga.gov.au" \\
                  'datacube[performance,s3]>=1.8.3.dev' \\
                  ${odc_libs}
                [ -f /build.info ] && cat /build.info

                aws s3 cp "${TASK_DB_S3}" tasks.db
                ls -lh tasks.db
                odc-stats run-pq --threads 16 --verbose tasks.db ${TASKS} {{extra}}
...
{% endfor %}""")

db_uri="s3://deafrica-stats-processing/s2_l2a_all.db"
docker="geoscienceaustralia/sandbox:latest"
extra="--public"
job="pq"

with open("stats-jobs.yaml", "wt") as f:
    f.write(tpl.render(N=100, start=0,
                       db_uri=db_uri,
                       docker=docker,
                       job=job,
                       extra=extra))